# Lab 5: Naive Bayes 

In this lab let's implement Naive Bayes by hand on some overly simplistic examples just so that we can get a sense of how the computations work and also some experience creating the vocabulary from plain text data.

## Tools

#### Libraries:

- pandas: for data processing  
- numpy: for processing and computations

#### Datasets: 

None

## Setup

In [1]:
# Import the only libraries we need 
import numpy as np
import pandas as pd

## Text Descriptions of Foods

Imagine you have 4 apples with these attributes

In [2]:
apples_docs = [
    "red round",
    "red round",
    "green sour round",
    "green round",
]

and 3 bananas with these attributes:

In [3]:
bananas_docs = [
    "yellow skinny",
    "yellow skinny",
    "green skinny"
]

Split both lists into a list of lists such that each apple and banana is in its own list within the larger list.

In [4]:
apples = [a.split() for a in apples_docs]
bananas = [b.split() for b in bananas_docs]

In [7]:
apples

[['red', 'round'],
 ['red', 'round'],
 ['green', 'sour', 'round'],
 ['green', 'round']]

---

**Q.** What is the sorted set of all attributes (assign to vocabulary variable $V$)?

Let's ignore the unknown word issue in our vectors and in our computations for now. Create a sorted list of all of the unique attributes of apples and bananas. 
### ['green', 'red', 'round', 'skinny', 'sour', 'yellow']

In [5]:
# your code here
Va = set(np.concatenate(apples))
Vb = set(np.concatenate(bananas))
V = sorted(Va.union(Vb))

In [6]:
Va

{'green', 'red', 'round', 'sour'}

In [8]:
V

['green', 'red', 'round', 'skinny', 'sour', 'yellow']

<details>
<summary>Solution</summary>
['green', 'red', 'round', 'skinny', 'sour', 'yellow']
    
You can compute like this:
    
```
Va = set(np.concatenate(apples))
Vb = set(np.concatenate(bananas))
V = sorted(Va.union(Vb))
```
</details>

---

**Q**. What is the word vector for the "red round" apple?

Now we have our vocabulary $V$, which you might think of as the column names of a document matrix where each row is a document. Here, the column values are the word counts, which will be 1 or 0 for our documents. Assume the sorted column order you've created above.

*your answer here*
### The row vector is [0, 1, 1, 0, 0, 0] for "red round

<details>
<summary>Solution</summary>
    The row vector is <tt>[0, 1, 1, 0, 0, 0]</tt> for "red round"
</details>

---

**Q**. What is the word vector for the "green sour round" apple?

The column values are 1 if the word is mentioned otherwise 0. Assume the sorted column order.

### The row vector is [1, 0, 1, 0, 1, 0] for "green sour round"

*your answer here*

<details>
<summary>Solution</summary>
    The row vector is <tt>[1, 0, 1, 0, 1, 0]</tt> for "green sour round"
</details>

---

Let's look at all fruit vectors now and fruit target column

In [9]:
data = np.zeros((7,len(V)))
for i,row in enumerate(apples+bananas):
    for w in row:
        data[i,V.index(w)] = 1
df = pd.DataFrame(data,columns=V,dtype=int)
df['fruit'] = [0,0,0,0,1,1,1]
df

,green,red,round,skinny,sour,yellow,fruit
0,0,1,1,0,0,0,0
1,0,1,1,0,0,0,0
2,1,0,1,0,1,0,0
3,1,0,1,0,0,0,0
4,0,0,0,1,0,1,1
5,0,0,0,1,0,1,1
6,1,0,0,1,0,0,1


In [17]:
columns=V
columns

['green', 'red', 'round', 'skinny', 'sour', 'yellow']

In [16]:
V.index('green')

0

In [12]:
df

,green,red,round,skinny,sour,yellow,fruit
0,0,1,1,0,0,0,0
1,0,1,1,0,0,0,0
2,1,0,1,0,1,0,0
3,1,0,1,0,0,0,0
4,0,0,0,1,0,1,1
5,0,0,0,1,0,1,1
6,1,0,0,1,0,0,1


---

**Q.** What is a good estimate of $P(apple)$=`P_apple` and $P(banana)$=`P_banana`?

(Define those variables)

In [18]:
# your code here
P_apple = 4/7
P_banana = 3/7
P_apple

0.42857142857142855

In [19]:
P_banana

0.42857142857142855

<details>
<summary>Solution</summary>
<pre>
P_apple = 4/7
P_banana = 3/7
</pre>
</details>

---

**Q.** What are good estimates of $P(red|apple)$ and $P(red|banana)$?

Count number of times red appears and divide by number of words in apple docs, then do for banana.

*your answer here*
### P(red|apple) = 2/9
### P(red|banana) = 0/6

<details>
<summary>Solution</summary>
P(red|apple) = 2/9 words in apples are red and 0/6 words in bananas are red.
</details>

---

**Q.** What are good estimates of $P(green|apple)$ and $P(green|banana)$?
### P(green|apple) = 2/9 
### P(green|banana) = 1/6

*your answer here*

<details>
<summary>Solution</summary>
2/9 apples are green and 1/6 bananas are green.
</details>

## Laplace smoothing of $P(w|c)$

**Q.** If $P(skinny|apple)=0$, what is our smoothed estimate? Do not include the "unknown" words in your estimate (leave off the +1 in the denominator).
### 𝑃(𝑠𝑘𝑖𝑛𝑛𝑦|𝑎𝑝𝑝𝑙𝑒)=(𝑤𝑜𝑟𝑑𝑐𝑜𝑢𝑛𝑡(𝑠𝑘𝑖𝑛𝑛𝑦,𝑎𝑝𝑝𝑙𝑒)+1)/(𝑤𝑜𝑟𝑑𝑐𝑜𝑢𝑛𝑡(𝑎𝑝𝑝𝑙𝑒)+|𝑉|)=(0+1)/(9+6)=.0666666

*your answer here*

<details>
<summary>Solution</summary>
$P(skinny|apple) = (wordcount(skinny,apple)+1)/(wordcount(apple)+|V|) = (0+1)/(9+6) = .0666666$
</details>

---

**Now, do the calculation you just did but using vector operations to get the smoothed estimate of $P(w|apple)$ for all $w$ from the apple records and call it `P_w_apple`**

Recall that `df[df.fruit==0]` gets you just the apple records.  Your $P(w|apple)$ resuls should be:

```
green     0.200000
red       0.200000
round     0.333333
skinny    0.066667
sour      0.133333
yellow    0.066667
fruit     0.066667
```

In [8]:
# your code here
w_counts_apple = df[df.fruit==0].sum(axis=0)
P_w_apple = (w_counts_apple+1) / (9+len(V))
P_w_apple

green     0.200000
red       0.200000
round     0.333333
skinny    0.066667
sour      0.133333
yellow    0.066667
fruit     0.066667
dtype: float64

<details>
<summary>Solution</summary>
    <pre>
w_counts_apple = df[df.fruit==0].sum(axis=0)
P_w_apple = (w_counts_apple+1) / (9+len(V))
P_w_apple
</pre>
</details>

---

**Do that same thing to estimate $P(w|banana)$ from the banana records and call it `P_w_banana`**

You should get:

```
green     0.166667
red       0.083333
round     0.083333
skinny    0.333333
sour      0.083333
yellow    0.250000
fruit     0.333333
```

In [9]:
# your code here
w_counts_banana = df[df.fruit==1].sum(axis=0)
P_w_banana = (w_counts_banana+1) / (6+len(V))
P_w_banana

green     0.166667
red       0.083333
round     0.083333
skinny    0.333333
sour      0.083333
yellow    0.250000
fruit     0.333333
dtype: float64

<details>
<summary>Solution</summary>
<pre>
w_counts_banana = df[df.fruit==1].sum(axis=0)
P_w_banana = (w_counts_banana+1) / (6+len(V))
P_w_banana
</pre>
</details>

---

**Q.** Given `P_w_apple`, what is `P_apple_redround`, the "probability" that "red round" is an apple, or better yet $P(apple | red, round)$?

(We haven't normalized the scores (per our friend Bayes) so they aren't technically probabilities.)  Just compute the score we'd use for classification per the lecture.  Hint: `P_w_apple['skinny']` gives the estimate of $P(skinny|apple)$. You can just use the product, no need to take the log for this one.

In [10]:
# your code here
P_apple_redround = P_apple * P_w_apple['red']*P_w_apple['round']
P_apple_redround

0.03809523809523809

<details>
<summary>Solution</summary>
    The answer is 0.03809523809523809 via:<br>
    <tt>P_apple_redround = P_apple * P_w_apple['red']*P_w_apple['round']</tt>
</details>

---

**Q.** Given `P_w_banana`, what is `P_banana_redround`, the "probability" that "red round" is a banana, or $P(banana|red,round)$?

In [11]:
# your code here
P_banana_redround = P_banana * P_w_banana['red']*P_w_banana['round']
P_banana_redround

0.002976190476190476

<details>
<summary>Solution</summary>
    The answer is 0.002976190476190476 via:<br>
    <tt>P_banana_redround = P_banana * P_w_banana['red']*P_w_banana['round']</tt>
</details>

Here's how to easily compute the probability of each word in V given class apple and class banana:

In [12]:
[P_w_apple[w] for w in V]

[0.2,
 0.2,
 0.3333333333333333,
 0.06666666666666667,
 0.13333333333333333,
 0.06666666666666667]

In [13]:
[P_w_banana[w] for w in V]

[0.16666666666666666,
 0.08333333333333333,
 0.08333333333333333,
 0.3333333333333333,
 0.08333333333333333,
 0.25]

---

**Now, define a function for computing the posterior, the $P(apple|d)$ for all of the documents $d$ in our dataframe, and then do the same for $P(banana|d)$**

$$
c^*= \underset{c}{argmax} ~ P(c) \prod_{w \in V} P(w | c)^{n_w(d)}
$$

You have these pieces: `P_apple`, `P_w_apple`, and $n_w(d)$ is just the value in `df[w][d]`.

In [14]:
def likelihood_apple(d:int):
    return P_apple * np.product([P_w_apple[w]**df[w][d] for w in V])
def likelihood_banana(d:int):
    return P_banana * np.product([P_w_banana[w]**df[w][d] for w in V])

<details>
<summary>Solution</summary>
<pre>
def likelihood_apple(d:int):
    return P_apple * np.product([P_w_apple[w]**df[w][d] for w in V])
def likelihood_banana(d:int):
    return P_banana * np.product([P_w_banana[w]**df[w][d] for w in V])
</pre>
</details>


---

**Run the following loop to make predictions for each document**

Output should be:

```
red round        : 0.038095, 0.002976 => apple
red round        : 0.038095, 0.002976 => apple
green sour round : 0.005079, 0.000496 => apple
green round      : 0.038095, 0.005952 => apple
yellow skinny    : 0.002540, 0.035714 => banana
yellow skinny    : 0.002540, 0.035714 => banana
green skinny     : 0.007619, 0.023810 => banana
```

In [15]:
docs = apples_docs+bananas_docs
for d in range(len(df)):
    a = likelihood_apple(d)
    b = likelihood_banana(d)
    print(f"{docs[d]:17s}: {a:4f}, {b:4f} => {'apple' if a>b else 'banana'}")

red round        : 0.038095, 0.002976 => apple
red round        : 0.038095, 0.002976 => apple
green sour round : 0.005079, 0.000496 => apple
green round      : 0.038095, 0.005952 => apple
yellow skinny    : 0.002540, 0.035714 => banana
yellow skinny    : 0.002540, 0.035714 => banana
green skinny     : 0.007619, 0.023810 => banana
